In [1]:
# !pip install pandas_datareader
import boto3
import base64
import sagemaker
from botocore.exceptions import ClientError
from IPython.display import display
import pandas_datareader
import pandas as pd
import numpy as np
from time import sleep
import os
import requests
import csv

  Using cached pandas_datareader-0.9.0-py3-none-any.whl (107 kB)


In [2]:
sess = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sess.default_bucket()
prefix = 'stock-data-raw'
tickers_url = 'http://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt'
# tickers = ['AAPL', 'NVDA'] ## to be changed to the entire list at http://ftp.nasdaqtrader.com/dynamic/SymDir/nasdaqlisted.txt

Couldn't call 'get_role' to get Role ARN from role name AmazonSageMaker-ExecutionRole-20210310T221946 to get Role path.
Assuming role was created in SageMaker AWS console, as the name contains `AmazonSageMaker-ExecutionRole`. Defaulting to Role ARN with service-role in path. If this Role ARN is incorrect, please add IAM read permissions to your role or supply the Role Arn directly.


In [3]:
## to do for massive data base; keeping small for now. 
def get_all_tickers(tickers_url):
    text = requests.get(tickers_url).text
    with open('temp.csv', 'w') as f:
        f.write(text)
    df = pd.read_csv('temp.csv', delimiter='|')
    tickers = df.Symbol.tolist()
    return tickers
tickers = get_all_tickers(tickers_url)
display(len(tickers))

4536

In [8]:
def get_secret():
    secret_name = "alpha_vantage"
    region_name = "us-east-2"
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )
    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except Exception as e:
        display(e)
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
        else:
            secret = base64.b64decode(get_secret_value_response['SecretBinary'])
    return secret   

def filter_complete_tickers(tickers, bucket, prefix):
    file_names = sess.list_s3_files(bucket, prefix)
    new = []
    for ticker in tickers:
        ticker_file_name = os.path.join(prefix, f'{ticker}_daily_raw.parquet')
        if ticker_file_name not in file_names:
            new.append(ticker)
    return new
                        
def read_data_to_s3(tickers, ALPHA_API_KEY, bucket, prefix):
    tickers = filter_complete_tickers(tickers, bucket, prefix)
    display('number of tickers', len(tickers))
    i = 0
    for ticker in tickers:      
        try: ## TODO: switch from error handling to simple check if exists; no pull of data
            pandas_datareader.av.time_series.AVTimeSeriesReader(symbols=ticker, api_key=ALPHA_API_KEY, function='TIME_SERIES_DAILY').read().to_parquet('temp.parquet')
            boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, f'{ticker}_daily_raw.parquet')).upload_file('temp.parquet')
            ## logic for api waiting
            i+=1
            if i%5==0: ## for 5 requests per minute limit on api
                display(f'waiting 61 seconds; {i} done')
                sleep(61)
            if i%500==0:
                display(f'waiting one day; {i} done')
                sleep(24*60*60+1) ## 1 day for 500 requests per day limit on api
        except ValueError as e:
            display(e)
        except Exception as e:
            display(e)
            display('waiting 61 seconds')
            sleep(61)
#         display(f'{ticker} done')       

In [9]:
ALPHA_API_KEY = get_secret()
len(filter_complete_tickers(tickers, bucket, prefix))

623

In [ ]:
read_data_to_s3(tickers, ALPHA_API_KEY, bucket, prefix)

'number of tickers'

623

ValueError('The requested symbol ATSPT could not be retrieved. Check valid ticker.')

ValueError('The requested symbol ATSPW could not be retrieved. Check valid ticker.')

ValueError('The requested symbol CMII could not be retrieved. Check valid ticker.')

...

'waiting 61 seconds'

pandas_datareader._utils.RemoteDataError(" Their was an issue from the data vendor side, here is their response: {'Information': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}")

'waiting 61 seconds'

pandas_datareader._utils.RemoteDataError(" Their was an issue from the data vendor side, here is their response: {'Information': 'Thank you for using Alpha Vantage! Our standard API call frequency is 5 calls per minute and 500 calls per day. Please visit https://www.alphavantage.co/premium/ if you would like to target a higher API call frequency.'}")

'waiting 61 seconds'

In [ ]:
## test whether upload worked
display(len(sess.list_s3_files(bucket, prefix)))
# sess.download_data('./test.parquet',bucket, prefix)

# df = pd.read_parquet('test.parquet')

# diplay(df)